In [7]:
%pip install -qU langchain_community beautifulsoup4

In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model='llama-3.1-70b-versatile'
)
response = llm.invoke('The first person to land on moon was ...')
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-49963?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer -II, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer -II, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. d

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from a job posting. Your job is to extract the following keys from the job posting in JSON format: 'role', 'experience', 'skills', 'description'.
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'page_data': page_data})
print(response.content)

```json
{
  "role": "Software Engineer -II, ITC",
  "experience": "2+ years of hands-on industry software development experience",
  "skills": [
    "Front-end frameworks like React, Angular, or Vue.js",
    "Cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools",
    "AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3",
    "Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest",
    "Architectural design patterns and computer-science fundamentals",
    "Implementing and integrating AI, Machine Learning and related data solutions",
    "Modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)"
  ],
  "description": "We’re looking for a Software Engineer to solve complex software engineering problems supporting Nike’s pursuit of delivering state of the art tools to our Consumer Product & Innovation community. The candidate needs to be highly collaborative with peers,

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_response = JsonOutputParser().parse(response.content)
json_response

{'role': 'Software Engineer -II, ITC',
 'experience': '2+ years of hands-on industry software development experience',
 'skills': ['Front-end frameworks like React, Angular, or Vue.js',
  'Cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
  'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
  'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
  'Architectural design patterns and computer-science fundamentals',
  'Implementing and integrating AI, Machine Learning and related data solutions',
  'Modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)'],
 'description': 'We’re looking for a Software Engineer to solve complex software engineering problems supporting Nike’s pursuit of delivering state of the art tools to our Consumer Product & Innovation community. The candidate needs to be highly collaborative with peers, productive in a fast-paced develo

In [14]:
type(json_response)

dict

In [15]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=["Experience with Python", "Experience with React"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [19]:
job = json_response
job['skills']

['Front-end frameworks like React, Angular, or Vue.js',
 'Cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
 'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
 'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
 'Architectural design patterns and computer-science fundamentals',
 'Implementing and integrating AI, Machine Learning and related data solutions',
 'Modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)']

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [21]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Heisenberg, a business development executive at Phetamine. Phetamine is an Software consulting company dedicated for seamless integration of business processes through automated tools.
    Over our experience, we have developed a strong portfolio of clients and projects.
    Your job is to write a cold email to the client regarding the job mentioned above descibing the capabilities of fulfilling their needs.
    Also add the most relevant links from the portfolio that you think will be helpful for the client. {link_list}
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
response = chain_email.invoke(input={'job_description': job['description'], 'link_list': links})
print(response.content)

Subject: Expert Software Engineering Solutions for Nike's Consumer Product & Innovation Community

Dear Hiring Manager,

I came across the job description for a Software Engineer at Nike, and I'm excited to introduce Phetamine, a software consulting company that can help fulfill your needs. With our expertise in seamless integration of business processes through automated tools, I believe we can support Nike's pursuit of delivering state-of-the-art tools to the Consumer Product & Innovation community.

Our team of skilled software engineers has a deep understanding of native cloud software engineering and can collaborate effectively with your peers to solve complex software engineering problems. We have a strong portfolio of clients and projects, and I'd like to highlight a few relevant examples that demonstrate our capabilities:

For frontend development, you can check out our TypeScript frontend portfolio: https://example.com/typescript-frontend-portfolio
Our expertise in DevOps can 